In [1]:
# mypy: disable-error-code="no-untyped-def, no-untyped-call"
# pyright: reportUnusedExpression=false, reportDuplicateImport=false, reportUnusedImport=false
import os
from pathlib import Path

p = Path.cwd()
for _ in range(100):
    if p.name == "cmomy":
        break
    p = p.parent

os.environ["NUMBA_CACHE_DIR"] = str(p / ".numba_cache")

In [2]:
%%html
<style>
    div.output_stderr {
    display: none;
    }
</style>


In [3]:
import numpy as np

np.set_printoptions(precision=4)

# Usage

The basic usage of cmomy is to manipulate central moments.  
We measure two quantities pretend quantities.  The 'energy' and the 'position' of a thing.  We'll construct the average value for each record. Lets say 100 samples each.  

In [4]:
import numpy as np
import xarray as xr

import cmomy
import cmomy.resample

rng = np.random.default_rng(seed=0)
cmomy.resample.numba_random_seed(0)

nsamp = 100
energy = xr.DataArray(rng.random(nsamp), dims=["samp"])

position = xr.DataArray(rng.random((nsamp, 3)), dims=["samp", "dim"])

# weight associated with each sample and each record
w = xr.DataArray(rng.random(nsamp), dims=["samp"])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim="samp", mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim="samp", mom=3)

In [5]:
print("energy")
ce

energy


<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])
Dimensions without coordinates: mom_0

In [6]:
print("position")
cp

position


<xCentralMoments(val_shape=(3,), mom=(3,))>
<xarray.DataArray (dim: 3, mom_0: 4)>
array([[ 5.3105e+01,  5.2437e-01,  6.9091e-02, -5.5768e-04],
       [ 5.3105e+01,  4.9062e-01,  9.5523e-02,  2.2967e-03],
       [ 5.3105e+01,  5.6372e-01,  8.7545e-02, -8.3042e-03]])
Dimensions without coordinates: dim, mom_0

Here, we've used the constructor {meth}`cmomy.CentralMoments.from_vals`.  There are a host of other constructors to create {class}`cmomy.CentralMoments` object.  Take a look at the docs for further info.

## Basic attributes

Notice that there are three `shape` parameters associated with a {class}`cmomy.CentralMoments` object:
 
* {attr}`cmomy.CentralMoments.mom_shape` : shape of the moments.  For single variable, tuple (mom+1,).  For comoments, (mom_0+1, mom_1+1)
* {attr}`cmomy.CentralMoments.val_shape`: shape of the 'values' part of the data.  For scalar data, `val_shape = ()`.  For vector data, this is the shape of the passed observation data.
* {attr}`cmomy.CentralMoments.shape`: total shape of wrapped moments `shape = val_shape + mom_shape`

In [7]:
for name, c in {"energy": ce, "position": cp}.items():
    print(
        f"""
{name} shapes:
    mom_shape: {c.mom_shape}
    val_shape: {c.val_shape}
    shape    : {c.shape}
"""
    )


energy shapes:
    mom_shape: (4,)
    val_shape: ()
    shape    : (4,)


position shapes:
    mom_shape: (4,)
    val_shape: (3,)
    shape    : (3, 4)



To access the underlying data, use the {meth}`cmomy.CentralMoments.to_numpy` method or {attr}`cmomy.CentralMoments.values` attribute.  The structure is:

* `values[...,0]`: total weight
* `values[...,1]`: average/mean value
* `values[...,k>1]`: `kth` central moment.

In [8]:
ce.to_numpy()

array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])

To access all the central moments (zeroth and first included), use the {meth}`cmomy.CentralMoments.cmom` method.

In [9]:
ce.cmom()

<xarray.DataArray (mom_0: 4)>
array([ 1.    ,  0.    ,  0.0864, -0.0057])
Dimensions without coordinates: mom_0

Likewise, the central moments can be converted to raw moments using the {meth}`cmomy.CentralMoments.rmom` method.  This uses the {mod}`cmomy.convert` module behind the scenes.  

In [10]:
# <x**k>
ce.rmom()

<xarray.DataArray (mom_0: 4)>
array([1.    , 0.5531, 0.3923, 0.3068])
Dimensions without coordinates: mom_0

Additionally, there are {class}`xarray.DataArray` like attributes

In [11]:
ce.coords

Coordinates:
    *empty*

In [12]:
ce.attrs

{}

In [13]:
ce.sizes

Frozen({'mom_0': 4})

In [14]:
ce.dims

('mom_0',)

To access the underlying data use {meth}`cmomy.xCentralMoments.to_dataarray` method or {attr}`cmomy.xCentralMoments.values` attribute to access the {class}`xarray.DataArray` style data, or {meth}`cmomy.xCentralMoments.to_numpy` method to access the underlying {class}`numpy.ndarray` data.b

In [15]:
ce.to_dataarray()

<xarray.DataArray (mom_0: 4)>
array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])
Dimensions without coordinates: mom_0

In [16]:
ce.to_numpy()

array([ 5.3105e+01,  5.5314e-01,  8.6378e-02, -5.7464e-03])

## Manipulating (co)moments

So we have our averages.  Cool.  Not very special.  But what if instead we repeat our experiment.  Let's say we did the experiment 10 times each time giving a single record.   Then our data would look like

In [17]:
nsamp = 100
nrec = 10
energy = xr.DataArray(rng.random((nrec, nsamp)), dims=["rec", "samp"])
position = xr.DataArray(rng.random((nrec, nsamp, 3)), dims=["rec", "samp", "dim"])

# weight associated with each sample and each record
w = xr.DataArray(rng.random((nrec, nsamp)), dims=["rec", "samp"])

# average over the samples
ce = cmomy.xCentralMoments.from_vals(x=energy, w=w, dim="samp", mom=3)
cp = cmomy.xCentralMoments.from_vals(x=position, w=w, dim="samp", mom=3)

Consider just the energy.  We suspect that there is some correlation between the experiments (they where done in rapid succession).  So we'd like to consider two adjacent experiments as one experiment.  For this, we can use the {meth}`cmomy.xCentralMoments.block` method.

In [18]:
ce

<xCentralMoments(val_shape=(10,), mom=(3,))>
<xarray.DataArray (rec: 10, mom_0: 4)>
array([[ 5.3064e+01,  4.7292e-01,  8.0077e-02,  6.5155e-03],
       [ 4.6744e+01,  4.7915e-01,  7.8450e-02,  1.8555e-03],
       [ 5.3254e+01,  5.0665e-01,  9.3529e-02,  2.1183e-03],
       [ 5.0303e+01,  5.3773e-01,  6.9736e-02, -6.0640e-03],
       [ 4.8231e+01,  5.2371e-01,  7.8316e-02, -3.9247e-03],
       [ 5.1988e+01,  4.4939e-01,  8.1112e-02,  3.2211e-03],
       [ 4.6908e+01,  4.9788e-01,  9.3582e-02, -4.0934e-03],
       [ 5.4791e+01,  4.8149e-01,  8.2545e-02,  8.0189e-04],
       [ 5.3189e+01,  4.6463e-01,  9.5055e-02,  3.3487e-03],
       [ 5.1267e+01,  5.4169e-01,  9.3554e-02, -4.9880e-03]])
Dimensions without coordinates: rec, mom_0

In [19]:
ce.block(block_size=2, dim="rec")

<xCentralMoments(val_shape=(5,), mom=(3,))>
<xarray.DataArray (rec: 5, mom_0: 4)>
array([[ 9.9808e+01,  4.7583e-01,  7.9325e-02,  4.3255e-03],
       [ 1.0356e+02,  5.2174e-01,  8.2213e-02, -2.4102e-03],
       [ 1.0022e+02,  4.8515e-01,  8.1146e-02, -3.6968e-04],
       [ 1.0170e+02,  4.8905e-01,  8.7702e-02, -1.3211e-03],
       [ 1.0446e+02,  5.0245e-01,  9.5802e-02, -8.2761e-04]])
Dimensions without coordinates: rec, mom_0

Instead, we can resample the already averaged data using {meth}`cmomy.xCentralMoments.resample_and_reduce`.  We produce a 20 new samples taken from the original (averaged) data.

In [20]:
ce_resamp = ce.resample_and_reduce(nrep=20, dim="rec")
ce_resamp

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 5.1549e+02,  5.0313e-01,  8.1351e-02, -1.2360e-03],
       [ 5.1124e+02,  4.7881e-01,  8.7858e-02,  8.4059e-04],
       [ 5.1466e+02,  4.9844e-01,  8.3805e-02, -2.9849e-04],
       [ 5.0900e+02,  5.0327e-01,  7.9809e-02, -1.3012e-03],
       [ 5.2328e+02,  5.0596e-01,  8.5539e-02,  1.4789e-04],
       [ 4.9984e+02,  4.9182e-01,  8.4933e-02, -1.0768e-04],
       [ 5.0926e+02,  5.0111e-01,  8.6995e-02, -8.3968e-04],
       [ 4.9676e+02,  5.1276e-01,  8.0943e-02, -2.7843e-03],
       [ 5.0676e+02,  4.8966e-01,  8.1455e-02, -1.0834e-04],
       [ 5.1316e+02,  4.9371e-01,  8.4144e-02,  1.2817e-03],
       [ 5.2884e+02,  4.9200e-01,  8.4514e-02,  1.9717e-03],
       [ 5.1017e+02,  5.1845e-01,  8.4040e-02, -1.8809e-03],
       [ 5.0293e+02,  4.9921e-01,  8.6330e-02, -2.1350e-04],
       [ 5.1695e+02,  4.9849e-01,  8.6562e-02, -7.5814e-04],
       [ 5.2097e+02,  4.9020e-01,  8.6304e-02,  6.9137e-04],
       [ 5.1247e+02,  4.9600e-01,  8.7239e-02, -2.8581e-05],
       [ 4.9987e+02,  5.0038e-01,  8.6787e-02, -1.2074e-03],
       [ 5.1595e+02,  4.9058e-01,  9.2541e-02,  9.3571e-05],
       [ 4.9801e+02,  5.0203e-01,  8.5986e-02, -2.5885e-03],
       [ 5.1690e+02,  4.8417e-01,  8.5900e-02,  7.5145e-04]])
Dimensions without coordinates: rep, mom_0

This is different than the usual 'resample values'.  This is also available if the original data is available.

In [21]:
a = energy.stack(c=["rec", "samp"])

In [22]:
b = w.stack(c=["rec", "samp"])

In [23]:
# consider 'all' the data for this
out = cmomy.xCentralMoments.from_resample_vals(
    energy.stack(c=["rec", "samp"]),
    w=w.stack(c=["rec", "samp"]),
    dim="c",
    nrep=20,
    mom=3,
)
out

<xCentralMoments(val_shape=(20,), mom=(3,))>
<xarray.DataArray (rep: 20, mom_0: 4)>
array([[ 5.1870e+02,  4.7400e-01,  8.4389e-02,  1.5778e-03],
       [ 5.1541e+02,  4.8502e-01,  8.6170e-02, -7.2000e-06],
       [ 5.1008e+02,  4.9256e-01,  9.0388e-02, -1.2888e-04],
       [ 5.1421e+02,  4.9885e-01,  8.2066e-02, -9.7701e-04],
       [ 5.0658e+02,  5.0539e-01,  8.8666e-02, -1.3791e-03],
       [ 5.1298e+02,  4.9358e-01,  8.4741e-02,  4.3089e-04],
       [ 4.9428e+02,  4.8285e-01,  8.3598e-02,  7.8853e-04],
       [ 5.0580e+02,  4.8493e-01,  7.9364e-02,  1.3492e-03],
       [ 5.0721e+02,  5.0514e-01,  8.4310e-02, -8.9118e-04],
       [ 5.1134e+02,  4.8263e-01,  8.8767e-02,  1.4280e-03],
       [ 5.0367e+02,  4.7357e-01,  8.6452e-02,  2.4973e-03],
       [ 5.0097e+02,  4.8563e-01,  8.9081e-02,  9.7469e-04],
       [ 5.0615e+02,  5.1440e-01,  8.1828e-02, -3.7775e-03],
       [ 4.9399e+02,  5.1711e-01,  8.2745e-02, -2.9780e-03],
       [ 5.0424e+02,  4.9504e-01,  8.8375e-02,  1.2170e-04],
       [ 5.0047e+02,  4.9491e-01,  8.7270e-02, -2.9224e-05],
       [ 5.2539e+02,  4.7645e-01,  8.6783e-02,  2.1366e-03],
       [ 4.9739e+02,  4.6122e-01,  8.3186e-02,  2.3753e-03],
       [ 5.1139e+02,  5.0466e-01,  8.4352e-02, -2.0612e-03],
       [ 4.9853e+02,  4.8582e-01,  8.5137e-02,  7.0537e-04]])
Coordinates:
    c        object (0, 0)
    rec      int64 0
    samp     int64 0
Dimensions without coordinates: rep, mom_0

We see that the deviation in the moments is similar using the two resampling methods:

In [24]:
out.to_dataarray().sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)>
array([0.0139, 0.0028, 0.0017])
Coordinates:
    c        object (0, 0)
    rec      int64 0
    samp     int64 0
Dimensions without coordinates: mom_0

In [25]:
ce_resamp.to_dataarray().sel(mom_0=slice(1, None)).std("rep")

<xarray.DataArray (mom_0: 3)>
array([0.009 , 0.0028, 0.0012])
Dimensions without coordinates: mom_0

We can also reduce our original data across all the records using {meth}`cmomy.xCentralMoments.reduce`

In [26]:
ce.reduce(dim="rec")

<xCentralMoments(val_shape=(), mom=(3,))>
<xarray.DataArray (mom_0: 4)>
array([ 5.0974e+02,  4.9508e-01,  8.5572e-02, -6.5659e-05])
Dimensions without coordinates: mom_0